In [ ]:
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

def reshape_to_4(array):
    return array.reshape(-1, 4)

train_predR = reshape_to_4(pd.read_csv('X_ensembleRSpred.csv').values)
train_predF = reshape_to_4(pd.read_csv('X_ensembleFSpred.csv').values)
train_LSTMpredR = reshape_to_4(pd.read_csv('X_ensembleRMpred.csv').values)
train_LSTMpredF = reshape_to_4(pd.read_csv('X_ensembleFMpred.csv').values)

val_predR = reshape_to_4(pd.read_csv('vrealMpred.csv').values)
val_predF = reshape_to_4(pd.read_csv('vfakeSpred.csv').values)
val_LSTMpredR = reshape_to_4(pd.read_csv('vrealMpred.csv').values)
val_LSTMpredF = reshape_to_4(pd.read_csv('vfakeMpred.csv').values)

test_predR = reshape_to_4(pd.read_csv('trealSpred.csv').values)
test_predF = reshape_to_4(pd.read_csv('tfakeSpred.csv').values)
test_LSTMpredR = reshape_to_4(pd.read_csv('trealMpred.csv').values)
test_LSTMpredF = reshape_to_4(pd.read_csv('tfakeMpred.csv').values)


In [ ]:
def get_spectrogram_stats(folder_path):
    stats = []
    zcr = []
    for filename in os.listdir(folder_path):
        audio, sr = librosa.load(os.path.join(folder_path, filename), sr=None)
        S = np.abs(librosa.stft(audio))
        S_db = librosa.amplitude_to_db(S, ref=np.max)

        mean_amp = np.mean(S_db)
        std_amp = np.std(S_db)
        max_amp = np.max(S_db)
        min_amp = np.min(S_db)

        rms = np.mean(librosa.feature.rms(y=audio))

        zc = np.sum(librosa.zero_crossings(audio)) / len(audio)

        #stats.append([mean_amp, std_amp, max_amp, min_amp, rms])
        stats.append([std_amp, max_amp, rms])
        zcr.append([zc])
    return np.array(stats), np.array(zcr)  

In [ ]:
real_path = r"\for-2seconds\training\real"
fake_path = r"\for-2seconds\training\fake"
real = []
fake = []

vreal_path = r"\for-2seconds\validation\real"
vfake_path = r"\for-2seconds\validation\fake"
vreal = []
vfake = []

treal_path = r"\for-2seconds\testing\real"
tfake_path = r"\for-2seconds\testing\fake"
treal = []
tfake = []


real_stats, real_zcr = get_spectrogram_stats(real_path)
real_zcr = real_zcr[-1744:]
real_stats = real_stats[-1744:]
fake_stats, fake_zcr = get_spectrogram_stats(fake_path)
fake_zcr = fake_zcr[-1744:]
fake_stats = fake_stats[-1744:]
vreal_stats, vreal_zcr = get_spectrogram_stats(vreal_path)
vfake_stats, vfake_zcr = get_spectrogram_stats(vfake_path)
treal_stats, treal_zcr = get_spectrogram_stats(treal_path)
tfake_stats, tfake_zcr = get_spectrogram_stats(tfake_path)

In [ ]:
real_stats.shape

In [ ]:
vreal_stats.shape

In [ ]:
all_stats = np.vstack([real_stats, fake_stats, vreal_stats, vfake_stats, treal_stats, tfake_stats])
scaler = StandardScaler().fit(all_stats)

real_stats = scaler.transform(real_stats)
fake_stats = scaler.transform(fake_stats)
vreal_stats = scaler.transform(vreal_stats)
vfake_stats = scaler.transform(vfake_stats)
treal_stats = scaler.transform(treal_stats)
tfake_stats = scaler.transform(tfake_stats)

In [ ]:
real_stats[1]

In [ ]:
X_train = np.vstack((np.hstack((train_predR, train_LSTMpredR, real_stats)),  
                          np.hstack((train_predF, train_LSTMpredF, fake_stats))))  

X_val = np.vstack((np.hstack((val_predR, val_LSTMpredR, vreal_stats)),  
                         np.hstack((val_predF, val_LSTMpredF, vfake_stats))))  

X_test = np.vstack((np.hstack((test_predR, test_LSTMpredR, treal_stats)),  
                         np.hstack((test_predF, test_LSTMpredF, tfake_stats))))  

zcr_train = np.vstack((real_zcr, fake_zcr))

zcr_val = np.vstack((vreal_zcr, vfake_zcr))

zcr_test = np.vstack((treal_zcr, tfake_zcr))

y_train = np.vstack((np.ones((train_predR.shape[0], 1)),  
                          np.zeros((train_predF.shape[0], 1))))  

y_val = np.vstack((np.ones((val_predR.shape[0], 1)),  
                          np.zeros((val_predF.shape[0], 1))))  

y_test = np.vstack((np.ones((test_predR.shape[0], 1)),  
                         np.zeros((test_predF.shape[0], 1))))  

In [ ]:
X_train = np.hstack((X_train, zcr_train))
X_val = np.hstack((X_val, zcr_val))
X_test = np.hstack((X_test, zcr_test))

In [ ]:
indices = np.arange(X_train.shape[0])
np.random.shuffle(indices)

X_train = X_train[indices]
y_train = y_train[indices]

In [ ]:
mlp_model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(16, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    
    Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),

    Dense(32, activation='relu'),
    BatchNormalization(),
    
    Dense(1, activation='sigmoid')
])
optimizer = Adam(learning_rate=0.0005)

mlp_model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])


mlp_model.fit(X_train, y_train, 
              epochs=20, batch_size=512, 
              verbose=2, validation_split=0.2)


mlp_accuracy1 = mlp_model.evaluate(X_val, y_val, verbose=0)[1]
mlp_accuracy2 = mlp_model.evaluate(X_test, y_test, verbose=0)[1]

print(f"Точність MLP на наборі з відомими синтезаторами: {mlp_accuracy1 * 100:.2f}%")
print(f"Точність MLP на наборі з невідомим синтезатором: {mlp_accuracy2 * 100:.2f}%")

In [ ]:
#mlp_model.save("mlp_model.keras")

In [ ]:
# from tensorflow.keras.models import load_model
# model = load_model('mlp_model.keras')

In [ ]:
#
# mlp_accuracy1 = model.evaluate(X_val, y_val, verbose=0)[1]
# mlp_accuracy2 = model.evaluate(X_test, y_test, verbose=0)[1]
# print(f"Точність MLP на схожих тестових: {mlp_accuracy1 * 100:.2f}%")
# print(f"Точність MLP на нових тестових: {mlp_accuracy2 * 100:.2f}%")

In [ ]:
# y_pred_prob = model.predict(X_test)

# y_pred = (y_pred_prob > 0.5).astype(int)

# cm = confusion_matrix(y_test, y_pred)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm)
# disp.plot(cmap=plt.cm.Blues)
# plt.show()

In [ ]:
svm_model = SVC(probability=True)
svm_model.fit(X_train, y_train.ravel())

svm_acc1 = svm_model.score(X_val, y_val)
svm_acc2 = svm_model.score(X_test, y_test)
print(f"Точність SVM на схожих тестових: {svm_acc1 * 100:.2f}%")
print(f"Точність SVM на нових тестових: {svm_acc2 * 100:.2f}%")

In [ ]:
# y_pred_prob = svm_model.predict(X_test)

# y_pred = (y_pred_prob > 0.5).astype(int)

# cm = confusion_matrix(y_test, y_pred)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["fake", "real"])
# disp.plot(cmap=plt.cm.Blues)
# plt.show()

In [ ]:
rf_model = RandomForestClassifier(n_estimators=10, max_depth=8, random_state=42)
rf_model.fit(X_train, y_train.ravel())

rf_acc1 = rf_model.score(X_val, y_val)
rf_acc2 = rf_model.score(X_test, y_test)
print(f"Точність Random Forest на схожих тестових: {rf_acc1 * 100:.2f}%")
print(f"Точність Random Forest на нових тестових: {rf_acc2 * 100:.2f}%")

In [ ]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'max_features': ['sqrt', 'log2', None]
}

rf = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                           cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

grid_search.fit(X_train, y_train.ravel())

best_rf_model = grid_search.best_estimator_

rf_acc1 = best_rf_model.score(X_val, y_val)
rf_acc2 = best_rf_model.score(X_test, y_test)

print("Найкращі параметри:", grid_search.best_params_)
print(f"Точність Random Forest на схожих тестових: {rf_acc1 * 100:.2f}%")
print(f"Точність Random Forest на нових тестових: {rf_acc2 * 100:.2f}%")


In [ ]:
# y_pred_prob = best_rf_model.predict(X_test)

# y_pred = (y_pred_prob > 0.5).astype(int)

# cm = confusion_matrix(y_test, y_pred)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["fake", "real"])
# disp.plot(cmap=plt.cm.Blues)
# plt.show()